# Пакеты `pandas` и `numpy`: типы данных


---

**Источники:**


[what are all the dtypes that pandas recognizes?](https://stackoverflow.com/questions/29245848/what-are-all-the-dtypes-that-pandas-recognizes)

[numpy.dtype](https://numpy.org/doc/stable/reference/generated/numpy.dtype.html#numpy.dtype)

[Data types](https://numpy.org/doc/stable/user/basics.types.html)

[Data type objects (dtype)](https://numpy.org/doc/stable/reference/arrays.dtypes.html#arrays-dtypes)

[Scalars](https://docs.scipy.org/doc/numpy-1.13.0/reference/arrays.scalars.html)

[Categorical data](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html)

[Essential basic functionality](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html)

[Extension types](https://pandas.pydata.org/pandas-docs/stable/development/extending.html#extending-extension-types)

---

## Подготовка окружения

In [1]:
# ВНИМАНИЕ: необходимо удостовериться, что виртуальная среда выбрана правильно!

!pip -V

pip 20.3.3 from /home/ira/anaconda3/envs/LevelUp_DataScience/lib/python3.8/site-packages/pip (python 3.8)


In [2]:
# !conda install pandas numpy lxml -y

In [3]:
import numpy as np

np.__version__

'1.19.2'

In [4]:
import pandas as pd

pd.__version__

'1.2.1'

## Обзор типов данных

**`Pandas` в основном использует массивы и типы `NumPy`** для каждой серии (фрейм данных - это набор серий, каждая из которых может иметь свой собственный `dtype`).

Документация `NumPy` дополнительно объясняет [`dtype`](https://numpy.org/doc/stable/reference/generated/numpy.dtype.html#numpy.dtype), [`Data types`](https://numpy.org/doc/stable/user/basics.types.html) и [`Data type objects (dtype)`](https://numpy.org/doc/stable/reference/arrays.dtypes.html#arrays-dtypes).


In [5]:
df = pd.DataFrame({'A': [1, 'C', 2.]})

df

,A
0,1
1,C
2,2.0


In [6]:
df['A']

0      1
1      C
2    2.0
Name: A, dtype: object

In [7]:
df['A'].dtype

dtype('O')

In [8]:
type(df['A'].dtype)

numpy.dtype

По умолчанию `int` - это `int64`, а `float` - `float64`, НЕЗАВИСИМО от платформы (32-разрядная или 64-разрядная).

In [9]:
pd.DataFrame([1, 2], columns=["a"]).dtypes

a    int64
dtype: object

In [10]:
pd.DataFrame({"a": [1, 2]}).dtypes

a    int64
dtype: object

In [11]:
pd.DataFrame({"a": 1}, index=list(range(2))).dtypes

a    int64
dtype: object

`Numpy`, однако, выберет платформо-зависимые типы *при создании массивов*. Следующее будет результатом int32 на 32-битной платформе.

In [12]:
pd.DataFrame(np.array([1, 2])).dtypes

0    int64
dtype: object

Одним из основных изменений в версии 1.0.0 `pandas` является введение `pd.NA` для представления скалярных пропущенных значений (а не предыдущих значений `np.nan`, `pd.NaT` или `None`, в зависимости от использования).

## Пакет `numpy`: типы данных


`Python` определяет только один тип определенного класса данных (существует только один целочисленный тип, один тип с плавающей запятой и т.д.).

Это может быть удобно в приложениях, которым не нужно заботиться обо всех способах представления данных на компьютере.

Однако для научных вычислений часто требуется больше контроля.


В `NumPy` есть `24` фундаментальных типа `Python` для описания различных типов скаляров.

Эти типы в основном основаны на типах, доступных в языке `C`, на котором написан `CPython`, с несколькими дополнительными типами, совместимыми с типами `Python`.

Хотя скаляры массива неизменяемы (immutable ), поэтому ни один из скалярных атрибутов массива не может быть установлен.

<img src="images/dtype_hierarchy.png"/>

In [13]:
def subdtypes(dtype):
    subs = dtype.__subclasses__()
    if not subs:
        return dtype
    return [dtype, [subdtypes(dt) for dt in subs]]

subdtypes(np.generic)

[numpy.generic,
 [[numpy.number,
   [[numpy.integer,
     [[numpy.signedinteger,
       [numpy.int8,
        numpy.int16,
        numpy.int32,
        numpy.int64,
        numpy.longlong,
        numpy.timedelta64]],
      [numpy.unsignedinteger,
       [numpy.uint8,
        numpy.uint16,
        numpy.uint32,
        numpy.uint64,
        numpy.ulonglong]]]],
    [numpy.inexact,
     [[numpy.floating,
       [numpy.float16, numpy.float32, numpy.float64, numpy.float128]],
      [numpy.complexfloating,
       [numpy.complex64, numpy.complex128, numpy.complex256]]]]]],
  [numpy.flexible,
   [[numpy.character, [numpy.bytes_, numpy.str_]],
    [numpy.void, [numpy.record]]]],
  numpy.bool_,
  numpy.datetime64,
  numpy.object_]]

In [14]:
np.float32(1.0)

1.0

In [15]:
np.int_([1,2,4])

array([1, 2, 4])

In [16]:
z = np.arange(3, dtype=np.uint8)
z

array([0, 1, 2], dtype=uint8)

Чтобы преобразовать тип массива, используйте метод `.astype()` (предпочтительно) или сам тип как функцию.

In [17]:
# предпочтительно
z = z.astype(float)  
z

array([0., 1., 2.])

In [18]:
z.dtype

dtype('float64')

In [19]:
z = np.complex_(z)
z

array([0.+0.j, 1.+0.j, 2.+0.j])

In [20]:
z.dtype

dtype('complex128')

In [21]:
d = np.dtype(int)
d

dtype('int64')

In [22]:
np.issubdtype(d, np.integer)

True

In [23]:
np.issubdtype(d, np.floating)

False

### [Ошибки переполнения (Overflow Errors)](https://numpy.org/doc/stable/user/basics.types.html#overflow-errors)

Поведение целочисленных типов `NumPy` и `Python` значительно отличается для целочисленных переполнений и может сбить с толку пользователей, ожидающих, что целые числа `NumPy` будут вести себя так же, как `int` в `Python`.

В отличие от `NumPy`, размер `int` в `Python` является гибким. Это означает, что **целые числа `Python` могут расширяться для размещения любого целого числа и не будут переполняться**.

**Фиксированный размер числовых типов `NumPy` может вызвать ошибки переполнения**, когда значение требует больше памяти, чем доступно в типе данных.

In [24]:
np.power(100, 8, dtype=np.int64)

10000000000000000

In [25]:
np.power(100, 8, dtype=np.int32)

1874919424

`NumPy` предоставляет `numpy.iinfo` и `numpy.finfo` для проверки минимального или максимального значений целых чисел и значений с плавающей запятой `NumPy` соответственно.

In [26]:
np.iinfo(int)

iinfo(min=-9223372036854775808, max=9223372036854775807, dtype=int64)

In [27]:
np.iinfo(np.int64)

iinfo(min=-9223372036854775808, max=9223372036854775807, dtype=int64)

In [28]:
np.iinfo(np.int32)

iinfo(min=-2147483648, max=2147483647, dtype=int32)

In [29]:
np.power(100, 100, dtype=np.int64)

0

In [30]:
np.power(100, 100, dtype=np.float64)

1e+200

## Пакет `Pandas`: типы данных

По большей части `pandas` использует массивы и типы `NumPy` для серий или отдельных столбцов `DataFrame`. 

`NumPy` обеспечивает поддержку `float`, `int`, `bool`, `timedelta64[ns]` и `datetime64[ns]` (обратите внимание, что `NumPy` не поддерживает дату и время с учетом часового пояса).

`pandas` и сторонние (third-party) библиотеки расширяют систему типов `NumPy` в нескольких местах.

В этом разделе описываются внутренние расширения `pandas`.

В следующей таблице перечислены все типы расширений `pandas`.

In [31]:
htmp_basic_info = 'https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#basics-dtypes'
basic_info_tables = pd.read_html(htmp_basic_info)

for index, table in enumerate(basic_info_tables):
    print(f"index = {index}")
    display(table)

In [32]:
pandas_types_df = basic_info_tables[3]

pandas_types_df

,Kind of Data,Data Type,Scalar,Array,String Aliases,Documentation
0,tz-aware datetime,DatetimeTZDtype,Timestamp,arrays.DatetimeArray,"'datetime64[ns, <tz>]'",Time zone handling
1,Categorical,CategoricalDtype,(none),Categorical,'category',Categorical data
2,period (time spans),PeriodDtype,Period,arrays.PeriodArray,"'period[<freq>]', 'Period[<freq>]'",Time span representation
3,sparse,SparseDtype,(none),arrays.SparseArray,"'Sparse', 'Sparse[int]', 'Sparse[float]'",Sparse data structures
4,intervals,IntervalDtype,Interval,arrays.IntervalArray,"'interval', 'Interval', 'Interval[<numpy_dtype...",IntervalIndex
5,nullable integer,"Int64Dtype, …",(none),arrays.IntegerArray,"'Int8', 'Int16', 'Int32', 'Int64', 'UInt8', 'U...",Nullable integer data type
6,nullable integer,"Int64Dtype, …",(none),arrays.IntegerArray,"'Int8', 'Int16', 'Int32', 'Int64', 'UInt8', 'U...",Nullable integer data type
7,Strings,StringDtype,str,arrays.StringArray,'string',Working with text data
8,Boolean (with NA),BooleanDtype,bool,arrays.BooleanArray,'boolean',Boolean data with missing values


### [Категориальные данные (Categorical data)](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html#categorical)

**Вид данных:** 

Категории - это тип данных, соответствующий категориальным переменным в статистике. 

Категориальная переменная принимает ограниченное и обычно фиксированное количество возможных значений.

Примеры: пол, социальный класс, группа крови, принадлежность к стране, язык.

### [Обработка часовых поясов (Time zone handling)](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-timezone)

**Вид данных:** 

Дата и время с учетом часового пояса (`tz`) (обратите внимание, что `NumPy` не поддерживает дату и время с учетом часовых поясов).

`pandas` предоставляет хорошую поддержку работы с отметками времен (timestamps) и в разных часовых поясах с использованием библиотек `pytz` и `dateutil` или объектов `datetime.timezone` из стандартной библиотеки.

`pandas` содержит обширные возможности и функции для работы с данными временных рядов для всех областей. 

Используя `datetime64` и `timedelta64` `NumPy`, `pandas` объединил большое количество функций из других библиотек `Python`, таких как `scikits.timeseries`, а также создал огромное количество новых функций для управления данными временных рядов.

Благодаря этому возможет анализ информации временных рядов из различных источников и форматов.

`pandas` представляет нулевые даты, временные интервалы и временные интервалы как `NaT,` что полезно для представления отсутствующих или нулевых значений даты, таких как значения, и ведет себя так же, как `np.nan` для данных с плавающей запятой.

`pandas` объединяет 4 общие концепции, связанные со временем:
- Дата и время: определенная дата и время с поддержкой часового пояса. Аналогично `datetime.datetime` из стандартной библиотеки.
- Дельты времени: абсолютная продолжительность времени. Аналогично `datetime.timedelta` из стандартной библиотеки.
- Интервалы времени: интервал времени, определяемый моментом времени и связанной с ним частотой.
- Смещения даты: относительная продолжительность времени, учитывающая календарную арифметику. Аналогично `dateutil.relativedelta.relativedelta` из пакета `dateutil`.

In [33]:
# список часовых поясов из пакета dateutil

from dateutil.zoneinfo import get_zonefile_instance

print(f"len = {len(get_zonefile_instance().zones)}")

get_zonefile_instance().zones

len = 595


{'Zulu': tzfile('Zulu'),
 'W-SU': tzfile('W-SU'),
 'Turkey': tzfile('Turkey'),
 'Singapore': tzfile('Singapore'),
 'ROK': tzfile('ROK'),
 'ROC': tzfile('ROC'),
 'Portugal': tzfile('Portugal'),
 'Poland': tzfile('Poland'),
 'PRC': tzfile('PRC'),
 'Navajo': tzfile('Navajo'),
 'NZ-CHAT': tzfile('NZ-CHAT'),
 'NZ': tzfile('NZ'),
 'Mexico/BajaNorte': tzfile('Mexico/BajaNorte'),
 'Mexico/BajaSur': tzfile('Mexico/BajaSur'),
 'Mexico/General': tzfile('Mexico/General'),
 'Libya': tzfile('Libya'),
 'Kwajalein': tzfile('Kwajalein'),
 'Japan': tzfile('Japan'),
 'Jamaica': tzfile('Jamaica'),
 'Israel': tzfile('Israel'),
 'Iran': tzfile('Iran'),
 'Iceland': tzfile('Iceland'),
 'Hongkong': tzfile('Hongkong'),
 'Greenwich': tzfile('Greenwich'),
 'GB-Eire': tzfile('GB-Eire'),
 'Eire': tzfile('Eire'),
 'Egypt': tzfile('Egypt'),
 'Cuba': tzfile('Cuba'),
 'Chile/Continental': tzfile('Chile/Continental'),
 'Chile/EasterIsland': tzfile('Chile/EasterIsland'),
 'Canada/Atlantic': tzfile('Canada/Atlantic'),
 'C

In [34]:
# список часовых поясов из пакета pytz

import pytz

print(f"len = {len(pytz.all_timezones)}")

pytz.all_timezones

len = 593


['Africa/Abidjan',
 'Africa/Accra',
 'Africa/Addis_Ababa',
 'Africa/Algiers',
 'Africa/Asmara',
 'Africa/Asmera',
 'Africa/Bamako',
 'Africa/Bangui',
 'Africa/Banjul',
 'Africa/Bissau',
 'Africa/Blantyre',
 'Africa/Brazzaville',
 'Africa/Bujumbura',
 'Africa/Cairo',
 'Africa/Casablanca',
 'Africa/Ceuta',
 'Africa/Conakry',
 'Africa/Dakar',
 'Africa/Dar_es_Salaam',
 'Africa/Djibouti',
 'Africa/Douala',
 'Africa/El_Aaiun',
 'Africa/Freetown',
 'Africa/Gaborone',
 'Africa/Harare',
 'Africa/Johannesburg',
 'Africa/Juba',
 'Africa/Kampala',
 'Africa/Khartoum',
 'Africa/Kigali',
 'Africa/Kinshasa',
 'Africa/Lagos',
 'Africa/Libreville',
 'Africa/Lome',
 'Africa/Luanda',
 'Africa/Lubumbashi',
 'Africa/Lusaka',
 'Africa/Malabo',
 'Africa/Maputo',
 'Africa/Maseru',
 'Africa/Mbabane',
 'Africa/Mogadishu',
 'Africa/Monrovia',
 'Africa/Nairobi',
 'Africa/Ndjamena',
 'Africa/Niamey',
 'Africa/Nouakchott',
 'Africa/Ouagadougou',
 'Africa/Porto-Novo',
 'Africa/Sao_Tome',
 'Africa/Timbuktu',
 'Africa/

In [35]:
import datetime

In [36]:
dti = pd.to_datetime(["1/1/2018",
                      np.datetime64("2018-01-01"), 
                      datetime.datetime(2018, 1, 1)])
dti

DatetimeIndex(['2018-01-01', '2018-01-01', '2018-01-01'], dtype='datetime64[ns]', freq=None)

In [37]:
dti = pd.date_range("2018-01-01", 
                    periods=3, 
                    freq="H")

dti

DatetimeIndex(['2018-01-01 00:00:00', '2018-01-01 01:00:00',
               '2018-01-01 02:00:00'],
              dtype='datetime64[ns]', freq='H')

In [38]:
# управление и преобразование даты и времени 
# с информацией о часовом поясе
dti.tz_localize("UTC")

DatetimeIndex(['2018-01-01 00:00:00+00:00', '2018-01-01 01:00:00+00:00',
               '2018-01-01 02:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='H')

In [39]:
dti.tz_localize("US/Pacific")

DatetimeIndex(['2018-01-01 00:00:00-08:00', '2018-01-01 01:00:00-08:00',
               '2018-01-01 02:00:00-08:00'],
              dtype='datetime64[ns, US/Pacific]', freq=None)

In [40]:
idx = pd.date_range("2018-01-01", periods=5, freq="H")
ts = pd.Series(range(len(idx)), index=idx)

ts

2018-01-01 00:00:00    0
2018-01-01 01:00:00    1
2018-01-01 02:00:00    2
2018-01-01 03:00:00    3
2018-01-01 04:00:00    4
Freq: H, dtype: int64

In [41]:
ts.resample("2H").mean()

2018-01-01 00:00:00    0.5
2018-01-01 02:00:00    2.5
2018-01-01 04:00:00    4.0
Freq: 2H, dtype: float64

In [42]:
friday = pd.Timestamp("2018-01-05")

friday.day_name()

'Friday'

In [43]:
# добавить 1 день
saturday = friday + pd.Timedelta("1 day")

saturday.day_name()

'Saturday'

In [44]:
# добавить 1 "бизнес" (рабочий) день
monday = friday + pd.offsets.BDay()

monday.day_name()

'Monday'

In [45]:
pd.Series(range(3), 
          index=pd.date_range("2000", freq="D", periods=3))

2000-01-01    0
2000-01-02    1
2000-01-03    2
Freq: D, dtype: int64

In [46]:
pd.Series(pd.date_range("2000", freq="D", periods=3))

0   2000-01-01
1   2000-01-02
2   2000-01-03
dtype: datetime64[ns]

In [47]:
pd.Series(pd.period_range("1/1/2011", freq="M", periods=3))

0    2011-01
1    2011-02
2    2011-03
dtype: period[M]

In [48]:
pd.Series([pd.DateOffset(1), pd.DateOffset(2)])

0         <DateOffset>
1    <2 * DateOffsets>
dtype: object

In [49]:
pd.Series(pd.date_range("1/1/2011", freq="M", periods=3))

0   2011-01-31
1   2011-02-28
2   2011-03-31
dtype: datetime64[ns]

In [50]:
pd.Timestamp(pd.NaT)

NaT

In [51]:
pd.Timedelta(pd.NaT)

NaT

In [52]:
pd.Period(pd.NaT)

NaT

In [53]:
# равенство действует так же, как np.nan
pd.NaT == pd.NaT

False

### [Представление промежутка времени (Time span representation)](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-periods)

**Вид данных:** 

Период представляет собой промежуток времени (например, день, месяц, квартал и т. Д.).

Можно указать диапазон с помощью ключевого слова `freq`.

Обычные интервалы времени представлены объектами `Period` в `pandas`, в то время как последовательности объектов `Period` собираются в `PeriodIndex`, который можно создать с помощью удобной функции `period_range`.

Данные с отметками времени (`Timestamp`) - это самый основной тип данных временных рядов, который связывает значения с моментами времени.

Для объектов `pandas` это означает использование точек во времени.

Однако во многих случаях более естественно связать такие вещи, как изменения переменных, с промежутком времени (time span / `Period`). Диапазон, представленный `Period` может быть указан явно или выведен из формата строки datetime.

In [54]:
pd.Timestamp(datetime.datetime(2012, 5, 1))

Timestamp('2012-05-01 00:00:00')

In [55]:
pd.Period("2011-01")

Period('2011-01', 'M')

In [56]:
pd.Period("2012-05", freq="D")

Period('2012-05-01', 'D')

Отметка времени и период могут служить индексом.

Списки `Timestamp` и `Period` автоматически приводятся к `DatetimeIndex` и `PeriodIndex` соответственно.

In [57]:
dates = [pd.Timestamp("2012-05-01"),
         pd.Timestamp("2012-05-02"),
         pd.Timestamp("2012-05-03"),]

ts = pd.Series(np.random.randn(3), dates)
ts

2012-05-01   -1.518771
2012-05-02    0.716577
2012-05-03   -0.211708
dtype: float64

In [58]:
type(ts.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [59]:
periods = [pd.Period("2012-01"), 
           pd.Period("2012-02"), 
           pd.Period("2012-03")]

ts = pd.Series(np.random.randn(3), periods)
ts

2012-01    1.658957
2012-02    0.306257
2012-03   -1.442650
Freq: M, dtype: float64

In [60]:
type(ts.index)

pandas.core.indexes.period.PeriodIndex

In [61]:
ts.index

PeriodIndex(['2012-01', '2012-02', '2012-03'], dtype='period[M]', freq='M')

Под капотом `pandas` представляет временные метки, используя экземпляры `Timestamp`, и последовательности временных меток, используя экземпляры `DatetimeIndex`.

Для регулярных интервалов времени `pandas` использует объекты `Period` для скалярных значений и `PeriodIndex` для последовательностей интервалов.

### [Разреженные структуры данных (Sparse data structures)](https://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#sparse)

**Вид данных:**

Различные разреженные матрицы.

Это не обязательно разреженность в типичном варианте: "в основном 0".

Скорее, можно рассматривать эти объекты как "сжатые", где любые данные, соответствующие определенному значению (`NaN` / отсутствующее значение, хотя может быть выбрано любое значение, включая `0`), опущены.

Сжатые значения фактически не хранятся в массиве.

### [Интервальный индекс (IntervalIndex)](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html#advanced-intervalindex)

**Вид данных:**

`IntervalIndex` вместе со своим собственным `dtype` - `IntervalDtype`, а также скалярным типом `Interval` обеспечивают первоклассную поддержку `pandas` для обозначения интервалов.


### [Обнуляемый целочисленный тип данных (Nullable integer data type)](https://pandas.pydata.org/pandas-docs/stable/user_guide/integer_na.html#integer-na)

**Вид данных:** 

`pandas` в основном использует `NaN` для представления отсутствующих данных.

Поскольку `NaN` является `float`, это заставляет массив целых чисел (`int`) с любыми пропущенными значениями стать `float`.

В некоторых случаях это может не иметь большого значения.

Но если целочисленный столбец является, например, идентификатором, приведение к типу `float` может быть проблематичным. 

Некоторые целые числа даже нельзя представить как числа с плавающей запятой.

### [Текстовые типы данных (Text data types)](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html)

**Вид данных:** 

Есть два способа хранить текстовые данные в `pandas`: 
- объект `dtype` `NumPy` `array`.
- тип `StringDtype`.


`pandas` рекомендует использовать `StringDtype` для хранения текстовых данных.

### [Логические данные с пропущенными значениями (Boolean data with missing values)](https://pandas.pydata.org/pandas-docs/stable/reference/arrays.html#api-arrays-bool)

**Вид данных:** 

Логический тип `dtype` (с псевдонимом `boolean`) обеспечивает поддержку для хранения логических данных (значения `True`, `False`) с пропущенными значениями, что невозможно с логическим значением `numpy.ndarray`.